In [0]:
oct_events = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
    header=True,
    inferSchema=True
)
nov_events = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
    header=True,
    inferSchema=True
)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:721)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:441)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:441)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
# Only use columns present in the table: x, y, z
df_duplicate = spark.table("workspace.ecommerce.Delta_oct_events").dropDuplicates(["x","y","z"])

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import current_timestamp

In [0]:
# Add ingestion metadata
bronze_df = oct_events.withColumn("ingestion_time", current_timestamp())

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-737187494129153>, line 2
      1 # Add ingestion metadata
----> 2 bronze_df = oct_events.withColumn("ingestion_time", current_timestamp())

NameError: name 'oct_events' is not defined

In [0]:
# Bronze Delta Path 
bronze_path = "workspace.default.events_delta"  

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:440)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:486)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:768)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:512)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:632)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$5(UsageLogging.scala:659)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:117)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:348)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:59)
	at com.databricks.logging.AttributionContext$.withValue(Att

In [0]:
%sql
create volume if not exists workspace.ecommerce.bronze;
create volume if not exists workspace.ecommerce.silver;
create volume if not exists workspace.ecommerce.gold;

In [0]:
# Add widgets for parameters
dbutils.widgets.text("source_path", "/Volumes/workspace/ecommerce/ecommerce_data")
dbutils.widgets.text("target_path", "/Volumes/workspace/ecommerce")
dbutils.widgets.dropdown("layer", "bronze", ["bronze","silver","gold"])

In [0]:
# Use parameters
from pyspark.sql.functions import lit, current_timestamp, col, to_date, sum, count, countDistinct
source_path = dbutils.widgets.get("source_path")
target = dbutils.widgets.get("target_path")
layer = dbutils.widgets.get("layer")
print(f"Running ETL for Layer: {layer} using source path: {source_path}")


Running ETL for Layer: bronze using source path: /Volumes/workspace/ecommerce/ecommerce_data


In [0]:
%sql
create catalog if not exists ecommerce;

In [0]:
%sql
use catalog ecommerce;

In [0]:
%sql
create volume if not exists bronze;
create volume if not exists silver;
create volume if not exists gold;

In [0]:
%sql
create schema if not exists bronze;
create table if not exists bronze.events(
    event_time timestamp,
    event_type string,
    product_id string,
    category_id string,
    category_code string,
    brand string,
    price string,
    user_id string,
    user_session string
)
using delta;

In [0]:
%sql
select  product_id, brand,
sum(revenue) as total_revenue
from gold.top_products
group by brand,product_id
order by total_revenue desc
limit 10;

product_id,brand,total_revenue
1005115,apple,6.606476334000108E7
1005105,apple,4.336920674000046E7
1004249,apple,2.70908150799991E7
1005135,apple,2.5308657539999034E7
1004767,samsung,2.200949697999872E7
1002544,apple,2.091779196000083E7
1004856,samsung,1.583586547999972E7
1005116,apple,1.4323876579999665E7
1002524,apple,1.3931065779999187E7
1004870,samsung,1.2114848060000682E7


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Revenue with 7-day moving average
WITH daily AS (
  SELECT brand,
     SUM(revenue) as rev
  FROM gold.product_metrics GROUP BY brand
)
SELECT brand, rev,
  AVG(rev) OVER (ORDER BY brand ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as ma7
FROM daily;

brand,rev,ma7


In [0]:
%sql
-- Conversion funnel
SELECT brand,
  SUM(views) as views,
  SUM(purchases) as purchases,
  ROUND(SUM(purchases)*100.0/SUM(views), 2) as conversion_rate
FROM gold.product_metrics
GROUP BY brand;

brand,views,purchases,conversion_rate


In [0]:
select 

In [0]:
%sql
SELECT
  CASE WHEN cnt >= 10 THEN 'VIP'
       WHEN cnt >= 5 THEN 'Loyal'
       ELSE 'Regular' END as tier,
  COUNT(*) as customers,
  AVG(total_spent) as avg_ltv
FROM (
  SELECT user_id, COUNT(*) cnt, SUM(price) total_spent
  FROM silver.events
  WHERE event_type='purchase' AND price IS NOT NULL
  GROUP BY user_id
)
GROUP BY tier;

tier,customers,avg_ltv


In [0]:
%sql
-- Customer tiers
SELECT
  CASE WHEN cnt >= 10 THEN 'VIP'
       WHEN cnt >= 5 THEN 'Loyal'
       ELSE 'Regular' END as tier,
  COUNT(*) as customers,
  AVG(total_spent) as avg_ltv
FROM (SELECT user_id, COUNT(*) cnt, SUM(price) total_spent
      FROM silver.events WHERE event_type='purchase' GROUP BY user_id)
GROUP BY tier;


tier,customers,avg_ltv
Regular,540822,676.927070126606
VIP,66452,7053.325390958875
Loyal,90196,1945.764277129805


Databricks visualization. Run in Databricks to view.

In [0]:
# Show logical plan for purchases in silver.events
df = spark.sql("select * from silver.events where event_type='purchase'")
df.explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('event_type = purchase)
   +- 'UnresolvedRelation [silver, events], [], false

== Analyzed Logical Plan ==
event_time: timestamp, event_type: string, product_id: string, category_id: string, category_code: string, brand: string, price: double, user_id: string, user_session: string
Project [event_time#14339, event_type#14340, product_id#14341, category_id#14342, category_code#14343, brand#14344, price#14345, user_id#14346, user_session#14347]
+- Filter (event_type#14340 = purchase)
   +- SubqueryAlias ecommerce.silver.events
      +- Relation ecommerce.silver.events[event_time#14339,event_type#14340,product_id#14341,category_id#14342,category_code#14343,brand#14344,price#14345,user_id#14346,user_session#14347] parquet

== Optimized Logical Plan ==
Filter (isnotnull(event_type#14340) AND (event_type#14340 = purchase))
+- Relation ecommerce.silver.events[event_time#14339,event_type#14340,product_id#14341,category_id#14342,category_code#14

In [0]:
%sql
create or replace table silver.events_part
using delta
partitioned by (event_date, event_type)
as 
select * , cast(event_time as date) as event_date
from silver.events;

num_affected_rows,num_inserted_rows


In [0]:
%sql
optimize silver.events_part
Zorder by(user_id, product_id);

path,metrics
,"List(65, 55, List(20995689, 110878338, 7.13485E7, 65, 4637652500), List(67651986, 159722740, 8.955611196363637E7, 55, 4925586158), 182, List(minCubeSize(107374182400), List(0, 0), List(188, 5997202311), 0, List(55, 4925586158), 49, null), null, 0, 1, 188, 133, false, 0, 0, 1768867635886, 1768867887673, 8, 49, null, List(0, 0), null, 10, 10, 456897, 0, null)"


In [0]:
# Benchmark
import time
start = time.time()
spark.sql("SELECT * FROM silver.events WHERE user_id=12345").count()
print(f"Time: {time.time()-start:.2f}s")

Time: 2.65s


In [0]:
benchmark = spark.sql("SELECT * FROM silver.events WHERE user_id=12345")


In [0]:
benchmark = spark.sql("SELECT * FROM silver.events_part WHERE user_id=12345")

In [0]:
# Cache for iterative queries
cached = spark.table("silver.events")
cached.count()  # Materialize


329852229

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import col, lit, to_date, sum, count, countDistinct
from pyspark.sql.functions import *


In [0]:
from pyspark.sql.window import Window

In [0]:
events = spark.table("ecommerce.silver.events")

In [0]:
events.describe("price").show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|         439802972|
|   mean| 291.6348023300816|
| stddev|356.67996969841073|
|    min|               0.0|
|    max|           2574.07|
+-------+------------------+



In [0]:
# Hypothesis: weekday vs weekend conversion
weekday = events.withColumn("is_weekend",
    F.dayofweek("event_time").isin([1,7]))
weekday.groupBy("is_weekend", "event_type").count().show()

+----------+----------+---------+
|is_weekend|event_type|    count|
+----------+----------+---------+
|     false|  purchase|  4186788|
|     false|      view|280916836|
|      true|      view|136425200|
|     false|      cart|  9854240|
|      true|      cart|  5967544|
|      true|  purchase|  2452364|
+----------+----------+---------+



In [0]:
# Correlation
# Only 'price' is a numeric column in events
# Correlation of 'price' with itself is always 1
# No other numeric column available for correlation

events = spark.table("ecommerce.silver.events")
events.stat.corr("price", "price")

1.0

In [0]:

# Feature engineering
features = events.withColumn("hour", F.hour("event_time")) \
    .withColumn("day_of_week", F.dayofweek("event_date")) \
    .withColumn("price_log", F.log(F.col("price")+1)) \
    .withColumn("time_since_first_view",
        F.unix_timestamp("event_time") -
        F.first("event_time").over(Window.partitionBy("user_id").orderBy("event_time")))